# SOM Visualization


## Description

This notebook contains code for visualizing the contents of hierarchical data (`*.h5`) files produced by [`Sompy_experimentation.ipynb`](Sompy_experimentation.ipynb). The current things it displays are as follows:

 - A set of linear-gradient heatmaps
 - A set of log-gradient heatmaps
 - An interactive interface for browsing through what items are in each cluster
 - A "U-matrix" that highlights regions of high euclidean distance gradients, to identify how well-matching the cluster boundaries are
 
 
 ## Workflow
 
 
 1. Enter the particular codebook file you want to use in [the codebook selection cell](#codebook_selection_cell), change the KM_CLUSTERS value to whatever number of clusters you feel is appropriate, and then rerun the notebook.
 1. Inspect the heatmaps in [the heatmap view cells](#heatmap_view_cell).
 1. Search the cells for materials of interest in [the cluster inspector cell](#cluster_inspector_cell).
 1. Evaluate the accuracy of the fit of the clusters in [the umatrix view cell](#umatrix_view_cell).
 
 
 ## Notes
 
  - The trained SOM has many arrays, one for each parameter it was trained on. This is what is seen in the heatmap views - each heatmap corresponds to one of these arrays.
  - The cluster inspector has various features, which shall be listed here.
      - First, each cluster is given its own tab in the inspector. The order of the tabs by color matches the pattern in the colorbar to the side of the cluster map. You can confirm the color of a given cluster with the colored square next to the cluster's name inside of the tab's frame.
      - When a cluster is selected, you may view its statistical information and contained items in the tables in the tab's frame.
      - You can restrict the number of items shown in the tab by entering a **regular expression**([1](https://en.wikipedia.org/wiki/Regular_expression), [2](https://www.regular-expressions.info/quickstart.html)) that matches the items you wish to see.
      - Once the items have been filtered to your liking, you can hit the "render points" button in order to display all items visible in the table to the cluster map.

In [1]:
import numpy as np
import pandas as pd
import logging
import ipywidgets as widgets
import matplotlib.pyplot as plt
import tables
import sompy
from sompy.sompy import SOMFactory

backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
# Codes from tfprop_sompy packages. Private code from Luna's group.
from tfprop_sompy.jupyter_integration.cluster_inspector import sort_materials_by_cluster, cluster_tabs,\
                                                                make_cluster_graph
from tfprop_sompy.tfprop_vis import render_posmap_to_axes, kmeans_clust, render_points_to_axes,\
                                    show_posmap, ViewTFP, dataframe_to_coords,\
                                    clusteringmap_category, UMatrixTFP

In [3]:
# This makes all the loggers stay quiet unless it's important
logging.getLogger().setLevel(logging.WARNING)

<a id='codebook_selection_cell'></a>

In [4]:
# Paste name of file generated by training in Sompy_experimentation
CODEBOOK_FILE = 'som_codemat_4props_20-02-21.h5'

# Number of clusters for k-means clustering
KM_CLUSTERS = 5

In [5]:
# Creates necessary pd dataframes for visualization
stored_cb_matrix = pd.read_hdf(CODEBOOK_FILE, 'sm_codebook_matrix')
stored_mapsize = pd.read_hdf(CODEBOOK_FILE, 'sm_codebook_mapsize').values
mats_df = pd.read_hdf(CODEBOOK_FILE, 'sm_data')

# FIXME:
# We do a hack using the "pytables" library in order to extract the information
# For some reason pandas doesn't like to read object series out of h5 files
stored_columns = None
stored_matfamilies = None
with tables.open_file(CODEBOOK_FILE, "r") as store:
    # We normally get byte strings from this
    # The mapping operation turns them all into unicode strings ready for presentation
    
    # Obtaining the column names
    stored_columns = list(map(lambda x: x.decode('utf-8'), store.root.sm_codebook_columns.property_names.read()))
    
    # Obtaining the material identification
    stored_matfamilies = list(map(lambda x: x, store.root.sm_codebook_matfamilies.material_families.read()))

FileNotFoundError: File som_codemat_4props_20-02-21.h5 does not exist

In [6]:
# Insert new column with material identification
mats_df["Row"] = stored_matfamilies

NameError: name 'stored_matfamilies' is not defined

In [8]:
# Function located in sompy.py
# Creates a SOM object
sm = SOMFactory.build(mats_df[stored_columns].values, 
                mapsize=(*stored_mapsize,),
                normalization="var", 
                initialization="pca", 
                component_names=stored_columns)

NameError: name 'mats_df' is not defined

In [9]:
# SOM object contains a codebook object
# Setting the matrix to the values of our current data. 
sm.codebook.matrix = stored_cb_matrix.values

NameError: name 'stored_cb_matrix' is not defined

In [10]:
# Using scikit-learn K-means clustering method to predict cluster fit index for our SOM object.
cl_labels = kmeans_clust(sm, KM_CLUSTERS)

NameError: name 'sm' is not defined

In [11]:
# Setting parameters for our heatmap
heatmap_size = (20, 20)
heatmap_col_sz = 4
gauss_alpha = None

cmap = plt.get_cmap('RdYlBu_r')  # set color map using Matplolib function

#Initializing a map viewer imported from mapview.py
viewTFP = ViewTFP(*heatmap_size, '', stdev_colorscale_coeff=1., text_size=14)

<a id='heatmap_view_cell'></a>

In [12]:
# Initializing widget
my_out = widgets.Output()

# No scaling
viewTFP.knee_value = 0.0

# Displaying heatmap through the widget
# Uses matplotlib functions
with my_out:
    print("Linear scaling")
    viewTFP.show(sm, cl_labels, col_sz=heatmap_col_sz,
                         which_dim='all', desnormalize=True, col_norm='mean',
                         cmap=cmap, isOutHtmap=False)
my_out

Output()

In [13]:
my_out = widgets.Output() # Initializing widget
cmap = plt.get_cmap('RdYlBu_r')  # set color map

# No scaling
viewTFP.knee_value = 0.0
with my_out:
    print("Log scaling")
    viewTFP.show(sm, cl_labels, col_sz=heatmap_col_sz,
                         which_dim='all', desnormalize=True, col_norm='mean',
                         cmap=cmap, normalizer="log")
my_out

Output()

In [14]:
mats_df.index

NameError: name 'mats_df' is not defined

<a id='cluster_inspector_cell'></a>

In [15]:
# Creating 2-D list organized by clusters
# 5 cluster = 5 lists within this list
clusters_list = sort_materials_by_cluster(sm, mats_df, cl_labels)

# This makes it so it will display the full lists
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# This should be the last statement of the cell, to make it display
# That, or assign the return value to a variable, and have that variable be the final expression in a cell
cluster_tabs(sm, mats_df, clusters_list, cl_labels)

NameError: name 'sm' is not defined

In [16]:
# Generating another cluster map to overlay material identification
fig, ax = make_cluster_graph(sm, cl_labels)

# Labels material identification on each data points on the cluster map
# Requires a column for color category
clusteringmap_category(ax, sm, KM_CLUSTERS, mats_df, "Row", mats_df["Row"], 'plot.png')

NameError: name 'sm' is not defined

<a id='umatrix_view_cell'></a>

In [17]:
# Initializing size
umat_size = (50, 50)

# Utilizes SOMPY's UMatrixView, which uses matplotlib
umat = UMatrixTFP(*umat_size, 'U-matrix')

# Plots the UMatrix map
# Must set "ifOutUmat" to True in order to output a image file
umat.show(sm, mats_df, mats_df, 'umatrix.png',cmap=cmap, isOutUmat=True)

NameError: name 'sm' is not defined

In [18]:
# Calculate quantization error value
sm.calculate_quantization_error()

NameError: name 'sm' is not defined

In [19]:
# Calculate topographic error value
sm.calculate_topographic_error()

NameError: name 'sm' is not defined